[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/klavis-ai/klavis/blob/main/examples/crewai/Salesforce_Gmail_CrewAI_Integration.ipynb)


# CrewAI with Klavis Salesforce + Gmail MCP Server

This tutorial demonstrates how to create a powerful AI crew that manages Salesforce opportunities and sends follow-up emails through Gmail using CrewAI with Klavis MCP servers.

## Use Case
- **Salesforce Agent**: Finds opportunities with pending next steps
- **Email Agent**: Drafts professional follow-up emails based on Salesforce data
- **Workflow**: Automated opportunity management and email communication


## Prerequisites

- **OpenAI API key** - Get at [openai.com](https://openai.com/)
- **Klavis API key** - Get at [klavis.ai](https://klavis.ai/)
- **Salesforce Account**
- **Gmail Account**


In [1]:
# Install the required packages
%pip install -qU crewai 'crewai-tools[mcp]' klavis

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os

# Set environment variables
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"  # Replace with your actual OpenAI API key
os.environ["KLAVIS_API_KEY"] = "YOUR_KLAVIS_API_KEY"  # Replace with your actual Klavis API key


## Step 1: Initialize Klavis Client

First, let's set up the Klavis client to create our MCP server instances.


In [4]:
from klavis import Klavis

klavis_client = Klavis(api_key=os.getenv("KLAVIS_API_KEY"))

✅ Klavis client initialized successfully


## Step 2: Create Salesforce MCP Server

Create a Salesforce MCP server instance and complete OAuth authentication.


In [36]:
import webbrowser
from klavis.types import McpServerName

# Create Salesforce MCP Server instance
salesforce_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.SALESFORCE,
    user_id="1234", 
    platform_name="Klavis" 
)
# Open OAuth URL in browser
webbrowser.open(salesforce_mcp_instance.oauth_url)

print(f"🔐 Opening OAuth authorization for Salesforce...")
# print(f"📱 If not redirected automatically, open: {salesforce_mcp_instance.oauth_url}")

🔐 Opening OAuth authorization for Salesforce...


## Step 3: Create Gmail MCP Server

Create a Gmail MCP server instance and complete OAuth authentication.


In [37]:
# Create Gmail MCP Server instance
gmail_mcp_instance = klavis_client.mcp_server.create_server_instance(
    server_name=McpServerName.GMAIL,
    user_id="1234",
    platform_name="Klavis"
)
# Open OAuth URL in browser
webbrowser.open(gmail_mcp_instance.oauth_url)

# print(f"🔗 Gmail MCP server created at: {gmail_mcp_instance.server_url}")
# print(f"🔐 Opening OAuth authorization for Gmail...")
# print(f"📱 If not redirected automatically, open: {gmail_mcp_instance.oauth_url}")

True

## Step 4: Configure MCP Server Adapters

Set up the MCP server adapters for CrewAI integration.


In [38]:
from crewai_tools import MCPServerAdapter
# Configure MCP server adapters
salesforce_mcp_server = MCPServerAdapter({
    "url": salesforce_mcp_instance.server_url,
    "transport": "streamable-http"
})

gmail_mcp_server = MCPServerAdapter({
    "url": gmail_mcp_instance.server_url,
    "transport": "streamable-http"
})


/Users/zihaolin/src/klavis/.venv/lib/python3.13/site-packages/pydantic/fields.py:1093: PydanticDeprecatedSince20: Using extra keyword arguments on `Field` is deprecated and will be removed. Use `json_schema_extra` instead. (Extra keys: 'items', 'anyOf', 'enum', 'properties'). Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  warn(


## Step 5: Create CrewAI Agents and Tasks

Define specialized agents for Salesforce opportunity management and email communication. Create specific tasks for opportunity analysis and email drafting.

In [39]:
from crewai import Agent, Task, Crew, Process
# Create Salesforce Opportunity Analyst Agent
salesforce_agent = Agent(
    role="Salesforce Opportunity Analyst",
    goal="Find opportunities with pending next steps and extract relevant contact information",
    backstory="You are an expert at analyzing Salesforce data to identify opportunities that need follow-up actions. You excel at extracting opportunity details, next step information, and contact data for effective sales management.",
    tools=salesforce_mcp_server.tools,    
    verbose=False,
)

# Create Email Communication Specialist Agent
email_agent = Agent(
    role="Email Communication Specialist",
    goal="Draft professional follow-up emails based on Salesforce opportunity next steps",
    backstory="You are an expert at crafting complete, professional follow-up emails without placeholder content. You always write proper signatures and complete email content based on Salesforce opportunity data and next step requirements.",
    tools=gmail_mcp_server.tools,    
    verbose=False,
)

# Define Salesforce Analysis Task
salesforce_task = Task(
    description="Find CrewAI opportunitity with next steps. Extract opportunity name, next step details, and contact emails. Focus on opportunities that require immediate follow-up actions.",
    expected_output="Comprehensive list of opportunities with next steps and contact information, including opportunity names, next step descriptions, contact emails, and priority levels",
    agent=salesforce_agent
)

# Define Email Drafting Task
email_task = Task(
    description="Based on the Salesforce opportunities analysis, draft professional follow-up emails for each opportunity. Include opportunity context, clear next actions, and professional tone. Ensure emails are complete and ready for sending.",
    expected_output="Draft emails with complete content ready for review and sending, including subject lines, professional greetings, opportunity context, next steps, and proper signatures",
    agent=email_agent,
    context=[salesforce_task]  # This task depends on the salesforce_task output
)

## Step 6: Execute the Crew

Create and run the multi-service crew to analyze Salesforce opportunities and draft follow-up emails.


In [40]:
try:
    # Create the crew
    salesforce_gmail_crew = Crew(
        agents=[salesforce_agent, email_agent],
        tasks=[salesforce_task, email_task],
        process=Process.sequential,
        verbose=True,  # Set to False to reduce output
    )
    # Execute the crew
    salesforce_gmail_crew.kickoff()
    
except Exception as e:
    print(f"❌ Error during crew execution: {e}")
    
finally:
    # Clean up resources
    salesforce_mcp_server.stop()
    gmail_mcp_server.stop()
    print("Resources cleaned up successfully")

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6f02408e-8092-49b9-b1ed-fdce83e0a944                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Salesforce Opportunity Analyst                                                                          │
│                                                                                                                 │
│  Task: Find CrewAI opportunitity with next steps. Extract opportunity name, next step details, and contact      │
│  emails. Focus on opportunities that require immediate follow-up actions.                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

/Users/zihaolin/src/klavis/.venv/lib/python3.13/site-packages/rich/live.py:231: UserWarning: install "ipywidgets" 
for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Salesforce Opportunity Analyst                                                                          │
│                                                                                                                 │
│  Thought: Action: salesforce_get_opportunities                                                                  │
│                                                                                                                 │
│  Using Tool: salesforce_get_opportunities                                                                       │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"name_contains\": \"CrewAI\", \"fields\": [\"Name\", \"NextStep\", \"AccountId\"]}"                         │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭────────────────────────────────────────────────── Tool Output ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Email draft created successfully with ID: r-6581095388079430028                                                │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Email Communication Specialist                                                                          │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  Subject: Follow-Up: Meeting Scheduling for CrewAI <> Klavis Opportunity                                        │
│                                                                                                                 │
│  Dear João Moura,                                                                                               │
│                                                                                                                 │
│  I hope this message finds you well. I am following up regarding our ongoing collaboration on the CrewAI <>     │
│  Klavis opportunity. I would like to schedule a meeting to discuss the next steps and further align our         │
│  efforts.                                                                                                       │
│                                                                                                                 │
│  Could we schedule the meeting for June 30th at 9 AM PST? Please let me know if this time works for you or if   │
│  there are any other slots that you would prefer.                                                               │
│                                                                                                                 │
│  Looking forward to your reply.                                                                                 │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│  [Your Position]                                                                                                │
│  [Your Company]                                                                                                 │
│  [Your Phone Number]                                                                                            │
│  [Your Email Address]                                                                                           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: f80c6d70-1715-4a60-ada0-2f7726d6b727                                                                     │
│  Agent: Email Communication Specialist                                                                          │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 6f02408e-8092-49b9-b1ed-fdce83e0a944                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: Subject: Follow-Up: Meeting Scheduling for CrewAI <> Klavis Opportunity                          │
│                                                                                                                 │
│  Dear João Moura,                                                                                               │
│                                                                                                                 │
│  I hope this message finds you well. I am following up regarding our ongoing collaboration on the CrewAI <>     │
│  Klavis opportunity. I would like to schedule a meeting to discuss the next steps and further align our         │
│  efforts.                                                                                                       │
│                                                                                                                 │
│  Could we schedule the meeting for June 30th at 9 AM PST? Please let me know if this time works for you or if   │
│  there are any other slots that you would prefer.                                                               │
│                                                                                                                 │
│  Looking forward to your reply.                                                                                 │
│                                                                                                                 │
│  Best regards,                                                                                                  │
│                                                                                                                 │
│  [Your Name]                                                                                                    │
│  [Your Position]                                                                                                │
│  [Your Company]                                                                                                 │
│  [Your Phone Number]                                                                                            │
│  [Your Email Address]                                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Resources cleaned up successfully


## Summary

This notebook demonstrated how to create a powerful Salesforce + Gmail integration using CrewAI with Klavis MCP servers.

### 🎯 **What We Accomplished:**

1. **Multi-Service Integration**: Connected Salesforce and Gmail through MCP servers
2. **Specialized Agents**: Created focused agents for opportunity analysis and email communication
3. **Sequential Workflow**: Implemented a workflow where Salesforce analysis feeds into email drafting
4. **OAuth Authentication**: Handled secure authentication for both services
5. **Professional Output**: Generated actionable insights and ready-to-send emails

### 🎓 **Next Steps:**
- Customize agent personalities and instructions for your use case
- Add more MCP servers for comprehensive workflow automation
- Implement scheduling and monitoring for production deployment
- Explore advanced CrewAI features like memory and planning

**Happy building!** 🚀📊📧
